# Importazione pacchetti e dati

## importare i pacchetti e i dati che servono: att run anche otto-nov-dic se si vogliono solo quei dati altrimenti è full

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import copy
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import random
import pickle

In [ ]:
# -----------------------------
# 1. Carica i dati
# -----------------------------
delivery_data = pd.read_csv('delivery_history.csv')

# -----------------------------
# 2. Rendiamo is_event un booleano sicuro
# -----------------------------
# Controlliamo che is_event abbia solo valori convertibili in booleano
valid_bool_mask = delivery_data['is_event'].isin([0, 1, True, False])
invalid_is_event = delivery_data.loc[~valid_bool_mask, 'is_event'].unique()
print("\n❌ Valori non convertibili in booleani in 'is_event':", invalid_is_event)

# Convertiamo solo i valori validi a booleano
delivery_data.loc[valid_bool_mask, 'is_event'] = delivery_data.loc[valid_bool_mask, 'is_event'].astype(bool)

# -----------------------------
# 3. Escludiamo righe dove is_event non è True
# -----------------------------
delivery_data = delivery_data[delivery_data['is_event'] == True].copy()

# -----------------------------
# 4. Filtra coordinate latitudine e longitudine (escludi points fuori range)
# -----------------------------
delivery_data = delivery_data[
    (delivery_data['lon'] >= 5) & (delivery_data['lon'] <= 12.5)
]

# -----------------------------
# 5. Escludiamo sabati e domeniche
# Supponendo che ci sia una colonna datetime 'date' o simile
# Se non c'è, bisognerebbe creare o specificare come avere il campo data
# -----------------------------
# Convertiamo la colonna data in datetime
delivery_data['delivery_date'] = pd.to_datetime(delivery_data['delivery_date'], errors='coerce')

# Drop righe con date non valide
delivery_data = delivery_data.dropna(subset=['delivery_date'])

# -----------------------------

# Prendiamo solo i giorni della settimana da lun (0) a ven (4)
delivery_data = delivery_data[delivery_data['delivery_date'].dt.dayofweek < 5]



# -----------------------------
# 6. Elimina duplicati per location_id lasciandone uno solo
# -----------------------------
delivery_points = delivery_data[['location_id', 'lat', 'lon']].drop_duplicates(subset='location_id').reset_index(drop=True)


# Stampa info per verifica
print(f"\nDataset preprocessato: {len(delivery_points)} punti univoci dopo filtri e rimozione weekend filtrando solo per ottobre, novembre e dicembre.")


## run questo sollo se si vuole i punti ottobre novembre dicembre

In [ ]:

# -----------------------------
# 6. Filtra solo ottobre,novembre,dicembre

# Filtra solo i mesi ottobre, novembre e dicembre
delivery_data = delivery_data[delivery_data['delivery_date'].dt.month.isin([10, 11, 12])]


# -----------------------------
# 6. Elimina duplicati per location_id lasciandone uno solo
# -----------------------------
delivery_points = delivery_data[['location_id', 'lat', 'lon']].drop_duplicates(subset='location_id').reset_index(drop=True)


# Stampa info per verifica
print(f"\nDataset preprocessato: {len(delivery_points)} punti univoci dopo filtri e rimozione weekend filtrando solo per ottobre, novembre e dicembre.")

# -----------------------------

## Run questo se si vuole i soli punti di Agosto-Settembre

In [ ]:

# -----------------------------
# 6. Filtra solo Agosto-settembre

# Filtra solo i mesi ottobre, novembre e dicembre
delivery_data = delivery_data[delivery_data['delivery_date'].dt.month.isin([8,9])]


# -----------------------------
# 6. Elimina duplicati per location_id lasciandone uno solo
# -----------------------------
delivery_points = delivery_data[['location_id', 'lat', 'lon']].drop_duplicates(subset='location_id').reset_index(drop=True)


# Stampa info per verifica
print(f"\nDataset preprocessato: {len(delivery_points)} punti univoci dopo filtri e rimozione weekend filtrando solo per ottobre, novembre e dicembre.")

# -----------------------------

## run questo per i soli punti di ottobre-novembre

In [ ]:

# -----------------------------
# 6. Filtra solo Ottobre-Novembre

# Filtra solo i mesi ottobre, novembre e dicembre
delivery_data = delivery_data[delivery_data['delivery_date'].dt.month.isin([10,11])]


# -----------------------------
# 6. Elimina duplicati per location_id lasciandone uno solo
# -----------------------------
delivery_points = delivery_data[['location_id', 'lat', 'lon']].drop_duplicates(subset='location_id').reset_index(drop=True)


# Stampa info per verifica
print(f"\nDataset preprocessato: {len(delivery_points)} punti univoci dopo filtri e rimozione weekend filtrando solo per ottobre, novembre e dicembre.")

# -----------------------------

# Analisi punti simili

## Metriche dati: location_id, freq & order_days

In [ ]:
metrics_df = (
    delivery_data.groupby('location_id')
    .agg(
        freq=('delivery_date', 'count'),
        order_days=('delivery_date', lambda x: tuple(sorted(x.dt.dayofweek.unique())))
    )
    .reset_index()
)

print(metrics_df.head())


# Codice per unire i punti con caratteristiche uguali

## Matrice delle distante in tempo

In [ ]:
def distance_matrix_creation_no_depot(delivery_points):
    """
    Calcola la matrice NxN di tempi in minuti tra tutti i punti di consegna (senza deposito).
    
    Parameters:
    - delivery_points: DataFrame con ['location_id', 'lat', 'lon']
    
    Returns:
    - distance_matrix_metri: matrice NxN distanze in metri (int)
    - time_mat_min: matrice NxN tempi di percorrenza in minuti (int)
    - index_to_location_id: lista indice->location_id
    - location_id_to_index: dict location_id->indice
    """

    points = delivery_points[['location_id', 'lat', 'lon']].reset_index(drop=True)

    R = 6371000.0  # raggio terrestre medio in metri
    lat = np.radians(points['lat'].values)
    lon = np.radians(points['lon'].values)

    dlat = lat[:, None] - lat[None, :]
    dlon = lon[:, None] - lon[None, :]

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat)[:, None] * np.cos(lat)[None, :] * np.sin(dlon / 2.0) ** 2
    c = 2.0 * np.arctan2(np.sqrt(a), np.sqrt(1.0 - a))
    distance_matrix_metri = (R * c).astype(int)

    dist_km = distance_matrix_metri.astype(float) / 1000.0

    speed_kmh = np.full_like(dist_km, 50.0)
    speed_kmh[dist_km <= 5.0] = 30.0
    speed_kmh[dist_km > 20.0] = 70.0

    np.fill_diagonal(speed_kmh, 1.0)  # evitare zero sulla diagonale

    time_min = (dist_km / speed_kmh) * 60.0
    np.fill_diagonal(time_min, 0.0)

    time_mat_min = np.ceil(time_min).astype(int)

    index_to_location_id = points['location_id'].tolist()
    location_id_to_index = {int(lid): i for i, lid in enumerate(points['location_id'])}

    return distance_matrix_metri, time_mat_min, index_to_location_id, location_id_to_index


# Supponendo di avere il DataFrame delivery_points con tutti i punti e colonne ['location_id', 'lat', 'lon']

# Calcolo matrice tempi e distanze
distance_matrix_metri, time_mat_min, index_to_location_id, location_id_to_index = distance_matrix_creation_no_depot(delivery_points)

# QUI si crea il mapping location_id -> indice rapidamente da usare nelle funzioni di clustering
location_id_to_index = {lid: idx for idx, lid in enumerate(index_to_location_id)}



## Calcolo della densità dei vari punti con divisione in base al quantile

In [ ]:
import numpy as np
import geopandas as gpd
from sklearn.neighbors import BallTree

# -----------------------------
# 1. Prepara i dati per analisi spaziale
# -----------------------------
# Trasformiamo i tuoi punti in un GeoDataFrame
gdf = gpd.GeoDataFrame(
    delivery_points,
    geometry=gpd.points_from_xy(delivery_points["lon"], delivery_points["lat"]),
    crs="EPSG:4326"   # lat/lon WGS84
)

# Convertiamo in metri (Web Mercator) per calcolare distanze reali
gdf = gdf.to_crs(epsg=3857)

# Estraggo coordinate in array
coords = np.array([(geom.x, geom.y) for geom in gdf.geometry])

# -----------------------------
# 2. Calcolo densità locale con BallTree
# -----------------------------
# Costruisco albero per ricerche veloci
tree = BallTree(coords, leaf_size=15, metric="euclidean")

# Raggio di vicinanza in metri (puoi modificarlo: es. 500 m)
raggio = 500  

# Conta quanti punti ci sono entro il raggio per ciascun punto
densita = tree.query_radius(coords, r=raggio, count_only=True)

# Aggiungo la colonna densità al GeoDataFrame
gdf["densita"] = densita

# -----------------------------
# 3. Classificazione bassa/media/alta densità
# -----------------------------
# Calcoliamo i quantili 33% e 66%
q1 = np.quantile(densita, 0.33)
q2 = np.quantile(densita, 0.66)

def classifica_densita(val):
    if val <= q1:
        return "bassa densità"
    elif val <= q2:
        return "media densità"
    else:
        return "alta densità"

gdf["classe"] = gdf["densita"].apply(classifica_densita)

# -----------------------------
# 4. Salvataggio / output
# -----------------------------
# Torniamo a EPSG:4326 per lat/lon
gdf = gdf.to_crs(epsg=4326)

# Esportiamo risultati
risultati = gdf[["location_id", "lat", "lon", "densita", "classe"]]
risultati.to_csv("delivery_points_densita.csv", index=False)

print("✅ Calcolo completato!")
print(risultati.head())
print(f"\nSoglie densità usate: q1={q1}, q2={q2}")


## Aggregazione dei punti tramite centroide

In [ ]:
def cluster_points_by_day_tuples_centroid(metrics_df, delivery_data, time_mat_min, location_id_to_index, density_classes):
    """
    Clustering con soglie di tempo diverse in base alla densità locale.

    density_classes: dict {location_id: 'bassa densità' | 'media densità' | 'alta densità'}
    """

    # Soglie per ciascuna classe di densità (in minuti)
    thresholds = {
        "bassa densità": 2,
        "media densità": 0.8,
        "alta densità": 0.2
    }

    cluster_list = []
    points_assigned = set()

    # Raggruppo i punti per giorno/i
    day_groups = metrics_df.groupby('order_days')['location_id'].apply(list)

    for day_tuple, loc_ids in day_groups.items():
        loc_ids_set = set(loc_ids)
        not_assigned = loc_ids_set - points_assigned

        while not_assigned:
            start_point = not_assigned.pop()
            cluster = {start_point}

            def compute_centroid(cluster):
                subset = delivery_data[delivery_data['location_id'].isin(cluster)]
                return subset['lat'].mean(), subset['lon'].mean()

            frontier = {start_point}

            while frontier:
                centroid_lat, centroid_lon = compute_centroid(cluster)
                cluster_points = list(cluster)
                cluster_indices = [location_id_to_index[p] for p in cluster_points]

                # Trova il punto del cluster più vicino al centroide
                center_idx = min(
                    cluster_indices,
                    key=lambda idx: np.sqrt(
                        (delivery_data.loc[idx, 'lat'] - centroid_lat) ** 2 +
                        (delivery_data.loc[idx, 'lon'] - centroid_lon) ** 2
                    )
                )

                # Determino la soglia in base alla densità del centro
                center_loc_id = delivery_data.loc[center_idx, 'location_id']
                dens_class = density_classes[center_loc_id]
                time_threshold = thresholds[dens_class]

                new_frontier = set()
                for q in not_assigned:
                    idx_q = location_id_to_index[q]
                    if time_mat_min[center_idx, idx_q] <= time_threshold:
                        cluster.add(q)
                        new_frontier.add(q)
                not_assigned -= new_frontier
                frontier = new_frontier

            cluster_list.append(list(cluster))
            points_assigned.update(cluster)

    # Ordina i cluster per dimensione decrescente
    cluster_list.sort(key=len, reverse=True)
    return cluster_list


In [ ]:
density_classes = dict(zip(gdf["location_id"], gdf["classe"]))

# --- 1. Esegui clustering (la tua funzione modificata) ---
clusters_centroid = cluster_points_by_day_tuples_centroid(
    metrics_df,
    delivery_points,
    time_mat_min,
    location_id_to_index,
    density_classes
)

# --- 2. Stampa i cluster e i punti ---
for i, cluster in enumerate(clusters_centroid, start=1):
    print(f"\nCluster {i} ({len(cluster)} punti):")
    for loc_id in cluster:
        # opzionale: mostra anche lat/lon
        point = delivery_points[delivery_points['location_id'] == loc_id].iloc[0]
        print(f"  {loc_id} -> lat: {point['lat']}, lon: {point['lon']}")

# --- 3. Salva in una variabile in memoria ---
# Dizionario: cluster_id -> lista location_id
clusters_dict = {i+1: cluster for i, cluster in enumerate(clusters_centroid)}

# Ora puoi accedere in memoria
print("\nVariabile clusters_dict pronta:")
print(clusters_dict)


## Analisi generali finali e salvataggio del file pickle

In [ ]:
print("Numero totale di clienti (punti di partenza):", len(delivery_points))
print("Numero totale di cluster (finali)", len(clusters_dict))
# Ordina i cluster per numero di punti (decrescente)
clusters_sorted = sorted(clusters_dict.items(), key=lambda x: len(x[1]), reverse=True)

# Prendi i primi 5 cluster
top5_clusters = clusters_sorted[:5]

print("Top 5 cluster con più punti:")
for cluster_id, points in top5_clusters:
    print(f"\nCluster {cluster_id} ({len(points)} punti): {points}")


In [ ]:
import pickle
import numpy as np

# Funzione per calcolare il punto di riferimento (quello più vicino al centroide)
def trova_punto_riferimento(cluster, delivery_points, location_id_to_index):
    subset = delivery_points[delivery_points['location_id'].isin(cluster)]
    centroid_lat = subset['lat'].mean()
    centroid_lon = subset['lon'].mean()

    # Trova il punto del cluster più vicino al centroide
    ref_id = min(
        cluster,
        key=lambda loc_id: np.sqrt(
            (delivery_points.loc[location_id_to_index[loc_id], 'lat'] - centroid_lat) ** 2 +
            (delivery_points.loc[location_id_to_index[loc_id], 'lon'] - centroid_lon) ** 2
        )
    )
    return ref_id

# Crea dizionario finale
clusters_pickle_dict = {}
for i, cluster in enumerate(clusters_centroid, start=1):
    ref_point = trova_punto_riferimento(cluster, delivery_points, location_id_to_index)
    clusters_pickle_dict[i] = (ref_point, cluster)

# Salva su file pickle
with open("clusters_output_punti_simili_OND.pkl", "wb") as f:
    pickle.dump(clusters_pickle_dict, f)

print("✅ File clusters_output.pkl salvato con successo!")


# Open file e Mappa dei soli cluster con almeno due punti

In [ ]:
# ----------------------
##########################
# Full
##########################
# ----------------------
percorso_file= 'clusters_output_punti_simili_ON.pkl'

# Apri il file pickle in lettura binaria
with open(percorso_file, 'rb') as f:
    clusters_pickle_dict = pickle.load(f)

# Stampa il contenuto del dizionario
print(clusters_pickle_dict)

# Conta i cluster
num_clusters = len(clusters_pickle_dict)

# Conta i punti totali (sommando la lunghezza delle liste di punti in ogni cluster)
total_points = sum(len(t[1]) for t in clusters_pickle_dict.values())

print('Numero cluster:', num_clusters)
print('Totale punti:', total_points)



In [ ]:
# Creiamo un dizionario location_id -> (lat, lon) per accesso rapido
loc_dict = delivery_points.set_index('location_id')[['lat', 'lon']].to_dict(orient='index')

lat_list = []
lon_list = []
colors = []

cluster_count = 0
points_count = 0

for cluster_id, (ref, points) in clusters_pickle_dict.items():
    if len(points) > 1:  # solo cluster con almeno 2 punti
        cluster_count += 1
        points_count += len(points)
        # Genera colore casuale per cluster
        color = (random.random(), random.random(), random.random())
        for pid in points:
            if pid in loc_dict:
                lat_list.append(loc_dict[pid]['lat'])
                lon_list.append(loc_dict[pid]['lon'])
                colors.append(color)

plt.figure(figsize=(10, 7))
plt.scatter(lon_list, lat_list, s=10, alpha=0.6, c=colors)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Points of Clusters with at least 2 Members ON')
plt.show()

print(f'Number of clusters with >1 point: {cluster_count}')
print(f'Total number of points in these clusters: {points_count}')
